In [1]:
# coding: utf-8
import ast
import json
import os
import sys
import re
from pprint import pprint
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime


from func import blast 

run_name = 'test'

PATH = os.path.abspath(".")

#################################
## Preparing Directories for Run

if not os.path.isdir('%s/Data/%s'%(PATH,run_name)):
    os.makedirs('%s/Data/%s'%(PATH,run_name))
    os.makedirs('%s/Data/%s/fragmented_fasta'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast/sh'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast/out'%(PATH,run_name))
    os.makedirs('%s/Data/%s/barcode_counts'%(PATH,run_name))
    os.makedirs('%s/Data/%s/db'%(PATH,run_name))


In [2]:
###############################
## Create BLAST database
databse   = "%s/test/test_database.csv"%(PATH) #Change this to your own database
const_seq = "%s/const-seq.csv"%(PATH) # DO NOT CHANGE THIS

db_fna    = blast.mkdb(databse,const_seq,run_name,"%s/Data/%s/db"%(PATH,run_name))

command = "makeblastdb -in %s  -dbtype nucl"%(db_fna)

print(os.popen(command).read())

Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna


Building a new DB, current time: 07/27/2021 17:53:46
New DB name:   /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna
New DB title:  /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/danyamamotoevans/GitHub/BFG/Data/test/db/test_database.fna
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 220 sequences in 0.00865102 seconds.



In [4]:
################################
## Convert fastq to fasta files in small chunks to make it easy to handle
miseq_dir =  #'FULL/PATH/TO/FASTQ' Put both R1 and R2 in the directory
blast.fastq2fasta(miseq_dir,150,"%s/Data/%s"%(PATH,run_name))

Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_1.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_2.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_3.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_4.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_5.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_6.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_7.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_001_8.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG/Data/test/fragmented_fasta/Undetermined_S0_L001_R1_00

In [5]:
#################################
## Create sh files to execute BLAST
blast.mk_blast_sh('%s/Data/%s/fragmented_fasta'%(PATH,run_name),db_fna, '%s/Data/%s'%(PATH,run_name))


In [ ]:
################################
## Execute BLAST
print("Executing BLAST for barcode alignment...(this takes time)\n")
print("Start time: %s"%datetime.datetime.now())
blast_sh = blast.get_sh('%s/Data/%s/blast/sh'%(PATH,run_name))

for sh in tqdm(blast_sh):
    command = "sh %s/Data/%s/blast/sh/%s"%(PATH,run_name,sh)
    os.system(command)
    os.system("rm %s/Data/%s/blast/sh/%s"%(PATH,run_name,sh))
#You can pause and re-execute this cell if it is partially done.

  0%|          | 0/52 [00:00<?, ?it/s]

Executing BLAST for barcode alignment...(this takes time)

Start time: 2021-07-27 18:12:35.336131


 62%|██████▏   | 32/52 [33:19<20:57, 62.87s/it]

In [ ]:
################################
## Parse BLAST output

print("Parsing BLAST alignment results...")

blast.parse_blast_out(PATH,run_name,db_fna)

        
print("\n\n\nProceed to BFG_data_normalization.ipynb to perform score normalization.")